In [1]:
import pandas as pd
import numpy as np

### Load the datasets

In [2]:
df_sum = pd.read_excel("1200_sampled_data.xlsx")
df_1 = pd.read_excel("Luke_posts.xlsx")
df_2 = pd.read_excel("Ada_posts.xlsx")
df_3 = pd.read_excel("Mare_posts.xlsx")
df_4 = pd.read_excel("Francesco_posts.xlsx")
df_final = pd.read_csv("raw_data.csv")

### Merge the datasets

In [4]:
df_combined = pd.concat([df_1, df_2, df_3, df_4], ignore_index=True)

In [5]:
df_merged = pd.merge(df_sum, df_combined, on = "pseudID", how = "left")

In [6]:
df = pd.merge(df_merged, df_final, left_on = "postID", right_on = "Post_ID", how = "right")

### Data cleaning

In [8]:
df.columns

Index(['date', 'text_x', 'pseudID', 'postID', 'text_y', 'StartDate', 'EndDate',
       'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished',
       'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName',
       'RecipientEmail', 'ExternalReference', 'LocationLatitude',
       'LocationLongitude', 'DistributionChannel', 'UserLanguage', 'Coder_ID',
       'Post_ID', 'Attribution_Presence', 'Attribution_Number',
       'Sender_Relevance_1', 'Evaluation_1', 'PresenceIndividual_1',
       'SpecificIndividual_1', 'PresenceCollective_1', 'SpecificCollective_1',
       'PresenceSystems_1', 'PresenceNetwork_1', 'Economy_1',
       'SocialPolicies_1', 'PoliticsHumanRight_1', 'ExternalRelations_1'],
      dtype='object')

In [9]:
# Drop irrelevant columns
cols_drop = df.loc[:, 'text_y':'UserLanguage'].columns
df = df.drop(columns=cols_drop)

df = df.drop(columns=["pseudID", "postID", "Coder_ID"])

In [10]:
# Drop irrelevant rows
df = df.iloc[2:] 

In [11]:
# Convert string "NaN" to missing values
df.replace("NaN", np.nan, inplace=True)

In [12]:
# sort the date from old to new
df = df.sort_values(by='date', ascending=True) 

In [13]:
# Create 'time_stamp' column based on the date range
# 2016-07-19 to 2017-01-19 (before Trump's first inauguration) -> 0
# 2017-01-20 to 2017-07-23 (after Trump's first inauguration) -> 1
df['Time_Stamp'] = df['date'].apply(lambda x: 0 if pd.Timestamp('2016-07-19') <= x <= pd.Timestamp('2017-01-19') else 1)

In [14]:
# function that codes yes as 1 and no as 0
def binary_encode_columns(df, columns):
    """
    Converts 'Yes' to 1 and 'No' to 0 in specified columns of a DataFrame.

    Parameters:
    - df: pandas DataFrame
    - columns: list of column names to convert

    Returns:
    - Modified DataFrame with binary encoding applied
    """
    for col in columns:
        df[col] = df[col].map({'Yes': 1, 'No': 0})
    return df

In [15]:
# Apply the function to columns with Yes and No options
df = binary_encode_columns(df, ['Attribution_Presence'])
df = binary_encode_columns(df, ['Sender_Relevance_1'])

In [16]:
# Recode values of Attribution_Number
df['Attribution_Number'] = df['Attribution_Number'].map({
    'First': 1,
    'Second': 2,
    'Third': 3,
    'Fourth': 4
})

In [17]:
# Recode values of Evaluation_1
df['Evaluation_1'] = df['Evaluation_1'].map({
    'Negatively': 0,
    'Positively': 1,
    'Ambiguously': None
})

In [18]:
# Combine actor columns
def assign_presence_code(row):
    if row['PresenceIndividual_1'] == "Yes":
        return 1
    elif row['PresenceCollective_1'] == "Yes":
        return 2
    elif row['PresenceSystems_1'] == "Yes":
        return 3
    elif row['PresenceNetwork_1'] == "Yes":
        return 4
    else:
        return np.nan

df["Target"] = df.apply(assign_presence_code, axis=1)

In [19]:
df = df.drop(columns=["PresenceIndividual_1", "PresenceCollective_1", "PresenceSystems_1", "PresenceNetwork_1"])

In [20]:
# Combine issue columns
def assign_issue_code(row):
    if row['Economy_1'] == "Yes":
        return 1
    elif row['SocialPolicies_1'] == "Yes":
        return 2
    elif row['PoliticsHumanRight_1'] == "Yes":
        return 3
    elif row['ExternalRelations_1'] == "Yes":
        return 4
    else:
        return np.nan

df["Topic"] = df.apply(assign_issue_code, axis=1)

In [21]:
df = df.drop(columns=["Economy_1", "SocialPolicies_1", "PoliticsHumanRight_1", "ExternalRelations_1"])

In [23]:
# Rename columns
df = df.rename(columns={
    'text_x': 'Text',
    'date': 'Date',
    'Sender_Relevance_1': 'Sender_Relevance',
    'Evaluation_1': 'Evaluation',
    'SpecificIndividual_1': 'Specific_Individual',
    'SpecificCollective_1': 'Specific_Collective'
})

In [24]:
# Reorder the columns
df = df[["Time_Stamp", "Date", "Text", "Post_ID", "Attribution_Presence", "Attribution_Number", "Sender_Relevance", "Evaluation", "Target", "Specific_Individual", "Specific_Collective", "Topic"]]

### Save the file

In [32]:
df.to_csv("clean_data.csv", index=False)